In [1]:
!pip install sentence_transformers
!pip install chromadb
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 3.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 14.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 55.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 82.1 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 85.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from sentence_transformers import SentenceTransformer
import pandas as pd
from chromadb.utils import embedding_functions
from tqdm import tqdm
import chromadb


In [3]:
!chroma run --path /db_path



                (((((((((    (((((####
             ((((((((((((((((((((((#########
           ((((((((((((((((((((((((###########
         ((((((((((((((((((((((((((############
        (((((((((((((((((((((((((((#############
        (((((((((((((((((((((((((((#############
         (((((((((((((((((((((((((##############
         ((((((((((((((((((((((((##############
           (((((((((((((((((((((#############
             ((((((((((((((((##############
                (((((((((    #########

    

Running Chroma

Saving data to: /db_path
Connect to chroma at: http://localhost:8000
Getting started guide: https://docs.trychroma.com/getting-started


INFO:     [20-02-2024 06:45:07] Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
DEBUG:    [20-02-2024 06:45:07] Starting component System
DEBUG:    [20-02-2024 06:45:07] Starting component OpenTelemetryClient
DEBUG:    [20-02-2024 06:45:07] Starting component SimpleAssi

In [4]:
client = chromadb.PersistentClient(path="/database/data")
print('База данных подключена') 
# invest_qa_emb = client.create_collection(name=db_name, metadata={'hnsw:space': 'cosine'})
collection = client.create_collection(name="my_collection", embedding_function=embedding_functions.SentenceTransformerEmbeddingFunction(model_name="intfloat/multilingual-e5-small"), metadata={'hnsw:space': 'cosine'})
print('Конфигурация коллекции создана')
df = pd.read_excel('/kaggle/input/data-with-pair-qa/data.xlsx')
collection.add(
    documents=df.Answer.to_list(),
    metadatas=[{'question': q} for q in df.Theme.to_list()],
    ids=[str(i) for i in range(len(df))]
)
print('Данные загружены')

База данных подключена


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

Конфигурация коллекции создана


Batches:   0%|          | 0/45 [00:00<?, ?it/s]

Данные загружены


In [17]:
collection = client.get_collection("my_collection")

embeddddd = SentenceTransformer("intfloat/multilingual-e5-small")
query = "Какие стратегии инвестирования есть?"
input_em = embeddddd.encode(sentences=f"query: {query}", batch_size=1).tolist()

results = collection.query(
    query_embeddings=[input_em],
    n_results=5
)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
answers = dict(zip(results['documents'][0], results['distances'][0]))
for key, value in answers.items():
    answers[key] = 1 - value

In [19]:
query = f'Вопрос пользователя "{query}'
answer = f'Ответы из базы знаний {answers}'

In [8]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

MODEL_NAME = "IlyaGusev/saiga_mistral_7b"
config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_8bit=False,
    torch_dtype=torch.float16,
    device_map="auto"
)
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)



adapter_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/623 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


generation_config.json:   0%|          | 0.00/265 [00:00<?, ?B/s]

GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 1536,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.1,
  "temperature": 0.2,
  "top_k": 40,
  "top_p": 0.9
}



In [15]:
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>"
DEFAULT_RESPONSE_TEMPLATE = "<s>bot\n"
DEFAULT_SYSTEM_PROMPT = "Ты Айаз Шабутдинов - интеллектуальный помощник по инвестициям. Ты должен вежливо отвечать на вопросы пользователей, основываясь на предоставленных ответах. Тебе будут предостоставлены ответы на похожие вопросы из базы знаний в формате (ответ: коэфициент релевантности вопросу пользователя). Если среди предоставленных ответов есть информация которая отвечает на вопрос, то составь красивый и понятный ответ для пользователя, а если среди ответов нет информации, которая отвечает на вопрос, то переспроси пользователя(уточни что он имел ввиду), ни в коем случае не придумывай информацию!"
class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        response_template=DEFAULT_RESPONSE_TEMPLATE
    ):
        self.message_template = message_template
        self.response_template = response_template
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })
    def add_answer_message(self, message):
        self.messages.append({
            "role": "answers",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += DEFAULT_RESPONSE_TEMPLATE
        return final_text.strip()
def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()

In [20]:
conversation = Conversation()
conversation.add_user_message(query)
conversation.add_answer_message(answer)
prompt = conversation.get_prompt(tokenizer)

output = generate(model, tokenizer, prompt, generation_config)
print()
print(output)
print("==============================")
print()


Существуют различные стратегии инвестирования, которые можно использовать в зависимости от целей, сроков и рисковых предпочтений инвестора. Одна из самых распространенных стратегий - это инвестирование в долгосрочные активы, такие как акции компаний с высоким потенциалом роста или фиксированные доходы. Другая стратегия - это инвестирование в компании с высокой стабильностью и надежностью, которые обеспечивают постоянный доход. Также существуют стратегии, основанные на специализации в определенной области, такой как технологии или здравоохранение. Важно помнить, что каждая стратегия имеет свои преимущества и недостатки, и следовательно, важно выбрать ту стратегию, которая наилучшим образом подходит для ваших инвестиционных целей и рисковых предпочтений.



In [24]:
!pip install python-telegram-bot

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 604.9/604.9 kB 10.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 5.7 MB/s eta 0:00:00


In [ ]:
import telebot

API_TOKEN = '7093361951:AAFDy4EkMv6pIB7MdlLsJKSKdLYcOVxcBjE'

bot = telebot.TeleBot(API_TOKEN)


# Handle '/start' and '/help'
@bot.message_handler(commands=['help', 'start'])
def send_welcome(message):
    bot.reply_to(message, output)


# Handle all other messages with content_type 'text' (content_types defaults to ['text'])
@bot.message_handler(func=lambda message: True)
def echo_message(message):
    bot.reply_to(message, message.text)


bot.infinity_polling()